# Лабораторная работа: Калькуляторы на Python с разбором выражений и `regex`

**Цель:** реализовать простой калькулятор для выражений без скобок с операторами `+ - * /`,
пошагово разобрав:
- как превратить строку в **токены** с помощью `regex`,
- как устроен простой **рекурсивный спуск** (`expr → term → factor`),
- почему токен числа представлен как кортеж `("NUM", float_value)` (и чем это удобно),
- как писать понятные ошибки и простые тесты.

> Важно: никакого `eval/exec` — всё делаем сами.



## 1. Постановка задачи и ограничения

- Вход: одна строка с выражением, например: `"3+4*2"`, `"-5+2*3"`, `" 10 / 4 "`.
- Поддерживаем **только**: `+ - * /` и **унарные** `+/-` перед числом. **Скобок нет.**
- Пробелы допускаются и игнорируются.
- Деление `/` — обычное вещественное (то есть `10/4 → 2.5`).

### Грамматика (EBNF)
```
expr   := term (('+'|'-') term)*
term   := factor (('*'|'/') factor)*
factor := ['+'|'-'] NUM
```
- Приоритеты: `* /` выше, чем `+ -`.
- Все бинарные операторы — **лево-ассоциативные** (слева направо).
- Унарный `+/-` обрабатывается на уровне `factor`.

## 2. Почему мы используем токены и что означает `("NUM", float(t))`

### 2.1. Зачем вообще токены?
Строка — это последовательность символов, но парсеру гораздо удобнее работать с **последовательностью смысловых единиц**:
чисел и операторов. Поэтому мы делаем **токенизацию** — превращаем строку в список **токенов**.

### 2.2. Как выглядит токен?
Мы выбрали очень простой и наглядный формат токена — **кортеж из двух элементов**:
- `token[0]` — **тип**, например: `"NUM"`, `"+"`, `"-"`, `"*"`, `"/"`, `"EOF"`;
- `token[1]` — **значение**, которое нужно только для чисел (для операторов оно не требуется).

Примеры токенов:
- Число 12.5 → `("NUM", 12.5)`
- Плюс → `("+", None)`
- Умножение → `"*", None"`

Мы **специально** делаем у всех токенов **одинаковую форму** `(тип, значение)`. Это упрощает код: парсер всегда ожидает кортеж из двух элементов.

### 2.3. Почему именно `("NUM", float(t))`?
- Мы **сразу** преобразуем строковое представление числа к числу типа `float`:
  - удобнее выполнять арифметику (не нужно каждый раз заново делать `float(...)`),
  - единый числовой тип в проекте (не нужно смешивать `int` и `float` и вспоминать, где какой).
- Альтернатива — хранить текст числа (`("NUM", "12.5")`) и преобразовывать позже. Но тогда нам пришлось бы:
  - конвертировать **многократно** (в каждом месте использования),
  - постоянно помнить, где уже float, а где ещё строка.
- Почему не `int`? Из-за деления `/` результат естественно может быть дробным. Держать всё в `float` проще для 1 курса.

> Итого: `("NUM", float_value)` — это **типобезопасно**, **коротко**, **удобно** для последующих вычислений.


## Регулярные выражения (кратко, без фанатизма)

Регулярные выражения (RegEx) — это язык шаблонов для поиска и обработки текста.


- `re.search(pattern, text)` — найти первое совпадение (или None).
- `re.findall(pattern, text)` — найти все совпадения (список строк).
- `re.finditer(pattern, text)` — найти все совпадения как объекты (с позициями).

Метасимволы

- `.` → любой символ, кроме \n
- `\d` → цифра ([0-9])
- `\w` → буква/цифра/подчёркивание
- `\s` → пробел, таб, перевод строки
- `^` → начало строки
- `$` → конец строки
- `[...]` → набор символов
- `[^...]` → отрицание


Квантификаторы

- `*` → 0 или больше
- `+` → 1 или больше
- `?` → 0 или 1
- `{n}` → ровно n
- `{n,}` → n или больше
- `{n,m}` → от n до m

Группы и захват

- `( )` → группа (захват текста)
- `(?: )` → группа без захвата
- `(?P<name>...)` → именованная группа

Флаги

- `re.IGNORECASE (re.I)` → без учёта регистра
- `re.MULTILINE (re.M)` → ^ и $ работают для каждой строки
- `re.DOTALL (re.S)` → . захватывает и \n

In [ ]:
import re

text = "Order #12345 on 2025-09-23 by user alice42"
m = re.search(r"#(\d+).*?(\d{4}-\d{2}-\d{2}).*?(\w+)$", text)
print("Номер заказа:", m.group(1), "дата:", m.group(2), "пользователь:", m.group(3))


Номер заказа: 12345 дата: 2025-09-23 пользователь: alice42


 Разбиение и замена

- `re.split(pattern, text)` — разбить строку по шаблону.
- `re.sub(pattern, repl, text)` — заменить все совпадения.

In [ ]:
print(re.split(r"\s+", "a   b\tc\nd"))
print(re.sub(r"\d+", "<NUM>", "id=42, port=8080"))

['a', 'b', 'c', 'd']
id=<NUM>, port=<NUM>


## 1. Разминка по `regex` (быстрый практикум)

Мы будем извлекать токены (числа и операторы) из строки. Основные техники:

- **Альтернация (`|`)**: позволяет перечислять варианты, например `\d+|\+|\-`.
- **Группы**: круглые скобки `(...)` — группируют подвыражение, можно извлекать `group(1)`, `group(2)`.
- **Ненумерованные группы `(?:...)`**: группировка без нумерации — удобно для читаемости.
- **Повторители**: `+`, `*`, `?`, `{m,n}` — число повторений.
- **Квантификаторы для чисел**: `\d+(?:\.\d+)?` — целое или десятичное.
- **Флаг `re.VERBOSE`**: можно разбивать паттерн на строки и комментировать его.
- **Ленивая/жадная квантификация** (нам почти не понадобится здесь).
- **Порядок вариантов имеет значение**: сначала **многосимвольные** операторы (`**`, `//`), потом односимвольные (`*`, `/`).


Мы хотим извлечь **числа** и **операторы**. Паттерн:

```regex
\s*                  # пропускаем пробелы перед каждым токеном
(
    \d+(?:\.\d+)?    # число: целое или с точкой, например 12 или 3.14
  | [+\-*/]          # один из операторов: + - * /
)
```

Ключевые идеи:
- `\s*` — игнорируем любые пробелы перед токеном;
- `\d+(?:\.\d+)?` — число: **обязательная целая часть** и **необязательная дробная** (`?:` — «незахватывающая» группа);
- `[+\-*/]` — любой из четырёх операторов;
- будем идти по строке и **по очереди** извлекать такие токены.


In [ ]:
import re

In [ ]:
text = "  12 + 3.5 * -2  "

In [ ]:
TOKEN_RE = re.compile(r"""
\s*
(
  [+-]?\d+(?:\.\d+)?
  | [+\-*/]
)
""", re.VERBOSE)

for index, op_match in enumerate(TOKEN_RE.finditer(text)):
  print("# match is {0}, match is {1}".format(index, op_match))
  for group in op_match.groups():
    print("Finded group: ", group)

# match is 0, match is <re.Match object; span=(0, 4), match='  12'>
Finded group:  12
# match is 1, match is <re.Match object; span=(4, 6), match=' +'>
Finded group:  +
# match is 2, match is <re.Match object; span=(6, 10), match=' 3.5'>
Finded group:  3.5
# match is 3, match is <re.Match object; span=(10, 12), match=' *'>
Finded group:  *
# match is 4, match is <re.Match object; span=(12, 15), match=' -2'>
Finded group:  -2


In [ ]:
(1 + 2) + (3 + 4 + (5 + 6))

In [ ]:
lst = [1,2,3,4,5,6]

for index, item in enumerate(lst):
  print(index, item)


0 1
1 2
2 3
3 4
4 5
5 6


Обратите внимание: унарный минус (например, в `* -2`) — это **не отдельный токен**, а **знак числа**.
Мы позже «склеим» его с числом на этапе предобработки (или обработаем в `factor()`).

### 1.1. Многосимвольные операторы: `**` и `//`

Если в языке есть `**` и `//`, **всегда** сначала выписываем их, затем односимвольные `*` и `/`,
иначе `re` захватит только первый символ и токенизация сломается.**Текст, выделенный полужирным шрифтом**

In [ ]:
import re

text = "2**3 + 7//3 - 4*2 / 5"
TOKEN_RE_EXT = re.compile(r"""
    \s*
    (
        \d+(?:\.\d+)?         # число
      | \*\*                   # ** (обязательно раньше *)
      | //                       # //
      | [%()+\-*/]              # одиночные токены
    )
""", re.VERBOSE)

tokens = [m.group(1) for m in TOKEN_RE_EXT.finditer(text)]
tokens


['2', '**', '3', '+', '7', '//', '3', '-', '4', '*', '2', '/', '5']

Иногда полезно видеть **позиции совпадений** для отладки токенизации.


In [ ]:
for m in TOKEN_RE_EXT.finditer(text):
    print(m.group(1), "at", (m.start(), m.end()))

2 at (0, 1)
** at (1, 3)
3 at (3, 4)
+ at (4, 6)
7 at (6, 8)
// at (8, 10)
3 at (10, 11)
- at (11, 13)
4 at (13, 15)
* at (15, 16)
2 at (16, 17)
/ at (17, 19)
5 at (19, 21)


## Функция `tokenize`: превращаем строку в список токенов

- Если символ не подходит ни под число, ни под оператор, бросаем `CalcError` с понятным сообщением.
- В конец добавим специальный токен `("EOF", None)` — это «конец файла/ввода». Парсеру удобно знать, что ввод закончился.


In [ ]:
12 // 0

ZeroDivisionError: integer division or modulo by zero

In [ ]:
TOKEN_RE = re.compile(r"\s*(\d+(?:\.\d+)?|[+\-*/])")

class CalcError(Exception):
    '''Понятные ошибки калькулятора.'''
    pass

Token = tuple[str, float | None]  # ("NUM", 12.5) или ("+", None) и т.д.



def tokenize(src: str) -> list[RECURSIVE]:
    '''
    Разбить строку на токены: числа и операторы.
    Для числа кладём ("NUM", float_value).
    Для оператора кладём (символ, None).
    В конец добавляем ("EOF", None).
    '''
    if not src or not src.strip():
        raise CalcError("Пустой ввод")

    pos = 0
    out: list[Token] = []

    while pos < len(src):
        m = TOKEN_RE.match(src, pos)
        if not m:
            # Покажем "хвост" строки, на котором застряли — это удобно при отладке.
            raise CalcError(f"Некорректный ввод около: '{src[pos:]}'")

        t = m.group(1)
        pos = m.end()

        # Если токен начинается с цифры — это число → сразу превращаем в float
        if t[0].isdigit():
            out.append(("NUM", float(t)))  # <-- Вот здесь тот самый ("NUM", float(t))
        else:
            out.append((t, None))

    out.append(("EOF", None))
    return out

tokenize("10 / 4 + -5 * 2")

[('NUM', 10.0),
 ('/', None),
 ('NUM', 4.0),
 ('+', None),
 ('-', None),
 ('NUM', 5.0),
 ('*', None),
 ('NUM', 2.0),
 ('EOF', None)]

In [ ]:
(1 + 2 + (4 +  (6 + 7)))


-> [6 + 7, 4 + ab, 1 + 2 + ac]


dictionary = {
    "6 + 7": "ab",
    "4 + ab": "ac"

}

## Что такое стек

Стек — это структура данных “последним пришёл — первым вышел” (LIFO: Last-In, First-Out). Работает как стопка тарелок: кладём сверху и снимаем тоже сверху.

Операции стека
- push(x) — положить элемент сверху
- pop() — снять верхний элемент
- peek() — посмотреть верхний элемент, не снимая
- is_empty() — пуст ли стек
- (опц.) size() — сколько элементов


Посмотрим как это реализовать на списке (`list`)

In [ ]:
def push(st: list[float], x: float) -> None:
    st.append(x)

def pop(st: list[float]) -> float:
    if not st:
        raise IndexError("pop from empty stack")
    return st.pop()

def peek(st: list[float]) -> float:
    if not st:
        raise IndexError("peek from empty stack")
    return st[-1]

def is_empty(st: list[float]) -> bool:
    return not st


In [ ]:
# Пример
st: list[float] = []
push(st, 1.5); push(st, 2.5); push(st, 3.5)  # [1.5, 2.5, 3.5]
print(peek(st))  # 3.5
print(pop(st))   # 3.5
print(pop(st))   # 2.5
print(is_empty(st))  # False
print(pop(st))   # 1.5
print(is_empty(st))  # True

3.5
3.5
2.5
False
1.5
True


Покажем на примере как обработать выражение с помощью стека

In [ ]:
import operator

OPS = {"+", "-", "*", "/"}

def calculate_rpn(expr: str) -> float:
    if not expr or not expr.strip():
        raise CalcError("Пустой ввод")

    st: list[float] = []
    for tok in expr.split():
        if tok in OPS:
            if len(st) < 2:
                raise CalcError("Недостаточно операндов")
            b = st.pop()       # второе число
            a = st.pop()       # первое число
            if tok == "+": st.append(a + b)
            elif tok == "-": st.append(a - b)
            elif tok == "*": st.append(a * b)
            elif tok == "/":
                if b == 0:
                    raise CalcError("Деление на ноль")
                st.append(a / b)
        else:
            try:
                st.append(float(tok))
            except ValueError:
                raise CalcError(f"Не число и не оператор: {tok}")

    if len(st) != 1:
        raise CalcError("Лишние данные в выражении")
    return st[0]


calculate_rpn("5 1 2 + 4 * + 3 -")

expr = "5 1 2 + 4 * + 3 -".split()


operations


stack = []


def plus(x,y):
  return x + y



def minus(x,y):
  return x - y



def multiply(x, y):
  return x * y


def division(x, y):
  return x / y


operations = {'+': operator.add, '-':operator.sub, "*": operator.mul, "/": operator.truediv }

for data in expr:
  print(data,stack)

  if data.isnumeric():
    stack.append(float(data))
  else:
    op1, op2 = stack.pop(), stack.pop()
    payload = operations[data](op1, op2)
    stack.append(payload)



print(stack)



5 []
1 [5.0]
2 [5.0, 1.0]
+ [5.0, 1.0, 2.0]
4 [5.0, 3.0]
* [5.0, 3.0, 4.0]
+ [5.0, 12.0]
3 [17.0]
- [17.0, 3.0]
[-14.0]


In [ ]:
stack = [1,2,3,4,5]

stack.pop(),stack

(5, [1, 2, 3, 4])

## Напишем парсер

Мы реализуем три функции в точности по грамматике. Каждая функция возвращает кортеж:
`(значение: float, новый_индекс: int)` — то есть **сколько мы съели токенов** и **какой результат получили**.

- `parse_expr(tokens, i)` вызывает `parse_term`, затем в цикле обрабатывает `+`/`-`,
- `parse_term(tokens, i)` вызывает `parse_factor`, затем в цикле обрабатывает `*`/`/`,
- `parse_factor(tokens, i)` разбирает **необязательный** унарный `+/-` и **обязательное** число `NUM`.


In [ ]:
def parse_factor(tokens: list[Token], i: int) -> tuple[float, int]:
    '''
    factor := ['+'|'-'] NUM
    Разрешаем унарный знак, затем ожидаем число ("NUM", float_value).
    '''
    sign = 1.0
    if tokens[i][0] in ('+', '-'):
        sign = 1.0 if tokens[i][0] == '+' else -1.0
        i += 1

    if tokens[i][0] != "NUM":
        raise CalcError("Ожидалось число")

    value = float(tokens[i][1])  # tokens[i] == ("NUM", float_value)
    return sign * value, i + 1


def parse_term(tokens: list[Token], i: int) -> tuple[float, int]:
    '''
    term := factor (('*'|'/') factor)*
    Выполняем умножения/деления по мере чтения (лево-ассоциативно).
    '''
    v, i = parse_factor(tokens, i)

    while tokens[i][0] in ('*', '/'):
        op = tokens[i][0]
        i += 1
        rhs, i = parse_factor(tokens, i)

        if op == '/':
            if rhs == 0:
                raise CalcError("Деление на ноль")
            v = v / rhs
        else:
            v = v * rhs

    return v, i


def parse_expr(tokens: list[Token], i: int) -> tuple[float, int]:
    '''
    expr := term (('+'|'-') term)*
    Складываем/вычитаем термы по мере чтения.
    '''
    v, i = parse_term(tokens, i)

    while tokens[i][0] in ('+', '-'):
        op = tokens[i][0]
        i += 1
        rhs, i = parse_term(tokens, i)
        v = v + rhs if op == '+' else v - rhs

    return v, i

# Дополнительно

## Lookahead / Lookbehind для унарных знаков и операторов

Тут будет показано как с помощью просмотров (lookaround) в `regex`:
- распознавать **унарные** `+/-` как часть числа;
- отличать `*` от `**` и `/` от `//` без путаницы;
- строить токенизацию, не усложняя парсер.

## 1. Числа с унарным знаком через lookbehind

Идея: число со знаком допустимо, если оно стоит **в начале строки** или **сразу после оператора/скобки**.  
В Python lookbehind должен иметь фиксированную длину, поэтому используем **три альтернативы**:

1) `(?<=^) [+-]?NUM` — в начале строки;  
2) `(?<=[+\-*/(]) [+-]?NUM` — после оператора или `(`;  
3) `(?<![\d)]) NUM` — обычное число, **не** после цифры или `)`.

Собираем единый шаблон с этими вариантами + операторы.

In [ ]:
import re

NUM = r"\d+(?:\.\d+)?"
SIGNED_AT_START = rf"(?<=^)[+\-]?{NUM}"
SIGNED_AFTER_OP = rf"(?<=[+\-*/(])[+\-]?{NUM}"
PLAIN_NUMBER    = rf"(?<![\d\)]){NUM}"

pattern = rf"""\s*(
  {SIGNED_AT_START}   |
  {SIGNED_AFTER_OP}   |
  \*\*              |
  //                  |
  [%()+\-*/]         |
  {PLAIN_NUMBER}
)
"""

TOKEN_RE = re.compile(pattern, re.VERBOSE)

def demo_tokens(s: str) -> list[str]:
    return [m.group(1) for m in TOKEN_RE.finditer(s)]

tests = [
    "-5+2*3",
    "+7- -2*3",
    "( -2 + 3 ) * 4",
    "2**3**2",
    "7//3 + 7%3",
    "12 / -3 * (+2)",
]
for t in tests:
    print(t, "=>", demo_tokens(t))

-5+2*3 => ['-5', '+', '2', '*', '3']
+7- -2*3 => ['+7', '-', '-', '2', '*', '3']
( -2 + 3 ) * 4 => ['(', '-', '2', '+', '3', ')', '*', '4']
2**3**2 => ['2', '**', '3', '**', '2']
7//3 + 7%3 => ['7', '//', '3', '+', '7', '%', '3']
12 / -3 * (+2) => ['12', '/', '-', '3', '*', '(', '+2', ')']



## 2. Отличаем одиночные `*` и `/` от `**` и `//` через отрицательный lookahead

- `\*(?!\*)` — одиночная `*` (за ней **не** идёт `*`),  
- `/(?!/)` — одиночный `/` (за ним **не** идёт `/`).

Это полезно, если вы хотите явно задать правило, а не полагаться на порядок альтернатив.

In [ ]:
import re

NUM = r"\d+(?:\.\d+)?"

pattern = rf"""
    \s*
    (
        \*\*            |   # двойная *
        //              |   # двойной /
        \*(?!\*)        |   # одиночная *
        /(?!/)          |   # одиночный /
        [+\-()%]        |   # одиночные токены: + - ( ) %
        {NUM}               # число
    )
"""

TOKEN_RE2 = re.compile(pattern, re.VERBOSE)

sample = "2**3 * 4 / 5 // 2 + 1"
print([m.group(1) for m in TOKEN_RE2.finditer(sample)])


['2', '**', '3', '*', '4', '/', '5', '//', '2', '+', '1']


In [ ]:
def parse(expr: str) -> list[str] | None:
  expr = expr.split()
  parsed = []

  for term in expr:
    if term.isnumeric():
      parsed.append(("NUM", float(term)))
    elif term in "+-*/":
      parsed.append(("OP", term))
    else:
      return None
  return parsed


In [ ]:
a = "123 + 456"
assert parse(a) == [("NUM", 123.0), ("OP", "+"), ("NUM", 456.0)]


b = "ab + 123"
assert parse(b) == None
